In [2]:
from google.colab import drive
drive.mount('/content/drive')
import os


Mounted at /content/drive


In [3]:
os.chdir("drive/MyDrive/work/research/other_misc/lablab/edgeNext/artificial-data-generation")

In [4]:
import numpy as np
import pandas as pd


In [16]:
! pip install peft


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [17]:
filename = "synData.csv"

df = pd.read_csv(filename,
                 names=["sentiment", "text"],
                 encoding="utf-8", encoding_errors="replace")

X_train = list()
X_test = list()
sentiments=df.sentiment.unique()
sentiment_string="\n ".join(sentiments)

print(sentiment_string)

positive
 spam
 neutral
 misleading


In [25]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import PeftModel, PeftConfig

import torch
def generate_test_prompt(data_point):
    return f"""The sentiment of the following phrase: '{data_point}' is \n{sentiment_string}\nCannot be determined
            \n\nSolution: The correct option is""".strip()

In [24]:
# Load the base model
model_name = "microsoft/Phi-3-mini-4k-instruct"  # Use the base model that was fine-tuned
base_model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

# Load the adapter configuration
adapter_dir = "trained-model"
adapter_config = PeftConfig.from_pretrained(adapter_dir)

# Load the adapter model and combine it with the base model
model = PeftModel.from_pretrained(base_model, adapter_dir)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Ensure the model is in evaluation mode
model.eval()


# inputs = tokenizer(input_text, return_tensors="pt").to(model.device)



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


TypeError: string indices must be integers

In [26]:
prompt="BREAKING: New Study Reveals 90% of People Who Drink Coffee Will Die from Heart Disease"
prompt=generate_test_prompt(prompt)
print(prompt)


The sentiment of the following phrase: 'BREAKING: New Study Reveals 90% of People Who Drink Coffee Will Die from Heart Disease' is 
positive
 spam
 neutral
 misleading
Cannot be determined
            

Solution: The correct option is


In [27]:

# Generate text with controlled length
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer)

# Use max_new_tokens instead of max_length
result = pipe(prompt, max_new_tokens=30, pad_token_id=pipe.tokenizer.eos_token_id)


The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM'

In [28]:
result

[{'generated_text': "The sentiment of the following phrase: 'BREAKING: New Study Reveals 90% of People Who Drink Coffee Will Die from Heart Disease' is \npositive\n spam\n neutral\n misleading\nCannot be determined\n            \n\nSolution: The correct option is misleading\nThe correct option is misleading because the phrase is a misleading headline that uses a sensationalist and exaggerated claim to"}]

In [29]:
answer = result[0]['generated_text'].split("The correct option is")[-1].lower()
selection="None"
for sentiment in sentiments:
    if sentiment in answer:
        selection=sentiment
        break

print("The category of the prompt is ",selection)

The category of the prompt is  misleading
